In [ ]:
from pathlib import Path
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix

# 1. Cargar dataset limpio

In [20]:
# Base del proyecto
base_dir = Path().resolve().parent
ruta_limpia = base_dir / "data" / "processed" / "upc_alumnos_limpio.csv"

# Cargar datos limpios
df = pd.read_csv(ruta_limpia)
print("✅ Dataset cargado:", df.shape)

✅ Dataset cargado: (72935, 12)


# 2. Feature engineering (conversión de texto a números)

In [21]:
# Longitud de campos
df['len_nombre'] = df['nombres'].astype(str).apply(len)
df['len_apellido_paterno'] = df['apellido_paterno'].astype(str).apply(len)
df['len_apellido_materno'] = df['apellido_materno'].astype(str).apply(len)

# Validación y binarización
df['telefono_valido'] = df['telefono_valido'].astype(int)
df['usa_correo_upc'] = df['usa_correo_institucional'].astype(int)

# Dominio de correo como variable categórica
df['dominio_correo'] = df['correo'].astype(str).apply(lambda x: x.split('@')[-1] if '@' in x else 'desconocido')

# 3. Codificación y selección de variables

In [25]:
# One-hot encoding para dominios de correo
ohe = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
dominios_encoded = ohe.fit_transform(df[['dominio_correo']])
dominios_df = pd.DataFrame(dominios_encoded, columns=ohe.get_feature_names_out(['dominio_correo']))

# Unir con el resto del dataset
X = pd.concat([
    df[['len_nombre', 'len_apellido_paterno', 'len_apellido_materno', 'telefono_valido']],
    dominios_df
], axis=1)

y = df['usa_correo_upc']

# 4. Modelo predictivo y evaluación

In [26]:
# División de datos
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Modelo
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

# Predicciones
y_pred = model.predict(X_test)

# Evaluación
print("✅ Reporte de clasificación:\n", classification_report(y_test, y_pred))
print("📊 Matriz de confusión:\n", confusion_matrix(y_test, y_pred))


✅ Reporte de clasificación:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00     14311
           1       0.95      0.98      0.97       276

    accuracy                           1.00     14587
   macro avg       0.98      0.99      0.98     14587
weighted avg       1.00      1.00      1.00     14587

📊 Matriz de confusión:
 [[14298    13]
 [    6   270]]
